In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import math

In [2]:
mpl.rcParams['figure.figsize'] = (32, 50)
mpl.rcParams['axes.grid'] = False

In [3]:
df = pd.read_csv(r'C:\Users\Левченко Никита\Downloads\Telegram Desktop\out.csv')

In [4]:
df

,A_bool_1,B_bool_1,P510_expense_A,P510_expense_B,Cu_AT501_A,Cd_AT501_A,Zn_AT501_A,Cu_AT501_B,Cd_AT501_B,Zn_AT501_B,...,temp_B,pH_А,pH_B,Cu_AT502_A,Cd_AT502_A,Zn_AT502_A,Cu_AT502_B,Cd_AT502_B,Zn_AT502_B,auto_W503_B
0,0.0,0.0,311.0,2.42,412.7,517.7,127.5,0.0,0.0,0.0,...,57.9,5.4,4.6,27.79,343.0,132.5,0.0,0.0,0.0,178.9
1,0.0,0.0,311.0,2.47,412.7,517.7,127.5,0.0,0.0,0.0,...,57.9,5.4,4.6,27.79,343.0,132.5,0.0,0.0,0.0,178.9
2,0.0,0.0,311.0,2.51,412.7,517.7,127.5,0.0,0.0,0.0,...,57.9,5.4,4.6,27.79,343.0,132.5,0.0,0.0,0.0,178.9
3,0.0,0.0,311.0,2.60,412.7,517.7,127.5,0.0,0.0,0.0,...,57.9,5.4,4.6,27.79,343.0,132.5,0.0,0.0,0.0,178.9
4,0.0,0.0,311.0,2.57,412.7,517.7,127.5,0.0,0.0,0.0,...,58.0,5.4,4.6,27.79,343.0,132.5,0.0,0.0,0.0,178.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253201,1.0,0.0,349.0,1.13,540.2,551.9,148.0,348.8,591.0,148.6,...,61.5,3.9,4.5,29.19,446.1,150.1,0.0,57.1,150.3,232.9
253202,1.0,0.0,349.0,1.13,540.2,551.9,148.0,348.8,591.0,148.6,...,61.5,3.9,4.5,29.19,446.1,150.1,0.0,57.1,150.3,232.9
253203,1.0,0.0,349.0,1.13,540.2,551.9,148.0,348.8,591.0,148.6,...,61.5,3.9,4.5,29.19,446.1,150.1,0.0,57.1,150.3,232.9
253204,1.0,0.0,349.0,1.13,540.2,551.9,148.0,348.8,591.0,148.6,...,61.5,3.9,4.5,29.19,446.1,150.1,0.0,57.1,150.3,232.9


In [5]:
# используемые параметры
features_considered = df.columns

In [6]:
features = df[features_considered]

In [7]:
#features.plot(subplots=True)

In [8]:
#features

In [9]:
# Удаление некорректных данных
need_rows = features[features['A_bool_1'] == 0.0]
need_rows = need_rows[need_rows['B_bool_1'] == 0.0]
features.drop(labels = need_rows.index, axis = 0, inplace = True)

need_rows = features[features['A_bool_1'] == 1.0]
need_rows = need_rows[need_rows['B_bool_1'] == 1.0]
features.drop(labels = need_rows.index, axis = 0, inplace = True)

need_rows = features[features['A_bool_2'] == 0.0]
need_rows = need_rows[need_rows['B_bool_2'] == 0.0]
features.drop(labels = need_rows.index, axis = 0, inplace = True)

need_rows = features[features['A_bool_2'] == 1.0]
need_rows = need_rows[need_rows['B_bool_2'] == 1.0]
features.drop(labels = need_rows.index, axis = 0, inplace = True)

In [10]:
# Вариант: в A и B происходят не взаимосвязанные химические процессы
# 4 массива
# A_bool_1 = 1; A_bool_2 = 1
# A_bool_1 = 1; B_bool_2 = 1
# B_bool_1 = 1; A_bool_2 = 1
# B_bool_1 = 1; B_bool_2 = 1

# A_bool_1 = 1
need_rows_A = features[features['A_bool_1'] == 1.0].copy(deep=True)
need_rows_A.drop(labels = ['A_bool_1',
                           'B_bool_1',
                           'P510_expense_B',
                           'Cu_AT501_B',
                           'Cd_AT501_B',
                           'Zn_AT501_B'], axis = 1, inplace = True)

# A_bool_1 = 1; A_bool_2 = 1
need_rows_A_A = need_rows_A[need_rows_A['A_bool_2'] == 1.0].copy(deep=True)
need_rows_A_A.drop(labels = ['A_bool_2',
                            'B_bool_2',
                            'temp_B',
                            'pH_B',
                            'Cu_AT502_B',
                            'Cd_AT502_B',
                            'Zn_AT502_B'], axis = 1, inplace = True)

# A_bool_1 = 1; B_bool_2 = 1
need_rows_A_B = need_rows_A[need_rows_A['B_bool_2'] == 1.0].copy(deep=True)
need_rows_A_B.drop(labels = ['A_bool_2',
                            'B_bool_2',
                            'temp_А',
                            'pH_А',
                            'Cu_AT502_A',
                            'Cd_AT502_A',
                            'Zn_AT502_A'], axis = 1, inplace = True)

# B_bool_1 = 1
need_rows_B = features[features['B_bool_1'] == 1.0].copy(deep=True)
need_rows_B.drop(labels = ['A_bool_1',
                           'B_bool_1',
                           'P510_expense_A',
                           'Cu_AT501_A',
                           'Cd_AT501_A',
                           'Zn_AT501_A'], axis = 1, inplace = True)

# B_bool_1 = 1; A_bool_2 = 1
need_rows_B_A = need_rows_B[need_rows_B['A_bool_2'] == 1.0].copy(deep=True)
need_rows_B_A.drop(labels = ['A_bool_2',
                            'B_bool_2',
                            'temp_B',
                            'pH_B',
                            'Cu_AT502_B',
                            'Cd_AT502_B',
                            'Zn_AT502_B'], axis = 1, inplace = True)


# B_bool_1 = 1; B_bool_2 = 1
need_rows_B_B = need_rows_B[need_rows_B['B_bool_2'] == 1.0].copy(deep=True)
need_rows_B_B.drop(labels = ['A_bool_2',
                            'B_bool_2',
                            'temp_А',
                            'pH_А',
                            'Cu_AT502_A',
                            'Cd_AT502_A',
                            'Zn_AT502_A'], axis = 1, inplace = True)

In [11]:
print('A A', need_rows_A_A.columns,
      '\n\nA B', need_rows_A_B.columns,
      '\n\nB A', need_rows_B_A.columns,
      '\n\nB B', need_rows_B_B.columns)

A A Index(['P510_expense_A', 'Cu_AT501_A', 'Cd_AT501_A', 'Zn_AT501_A', 'temp_А',
       'pH_А', 'Cu_AT502_A', 'Cd_AT502_A', 'Zn_AT502_A', 'auto_W503_B'],
      dtype='object') 

A B Index(['P510_expense_A', 'Cu_AT501_A', 'Cd_AT501_A', 'Zn_AT501_A', 'temp_B',
       'pH_B', 'Cu_AT502_B', 'Cd_AT502_B', 'Zn_AT502_B', 'auto_W503_B'],
      dtype='object') 

B A Index(['P510_expense_B', 'Cu_AT501_B', 'Cd_AT501_B', 'Zn_AT501_B', 'temp_А',
       'pH_А', 'Cu_AT502_A', 'Cd_AT502_A', 'Zn_AT502_A', 'auto_W503_B'],
      dtype='object') 

B B Index(['P510_expense_B', 'Cu_AT501_B', 'Cd_AT501_B', 'Zn_AT501_B', 'temp_B',
       'pH_B', 'Cu_AT502_B', 'Cd_AT502_B', 'Zn_AT502_B', 'auto_W503_B'],
      dtype='object')


In [12]:
_columns = list(need_rows_A_A.columns)
for i in range(len(_columns)):
    _columns[i] = _columns[i][:-2]
need_rows_A_A.columns = _columns

_columns = list(need_rows_A_B.columns)
for i in range(len(_columns)):
    _columns[i] = _columns[i][:-2]
need_rows_A_B.columns = _columns

_columns = list(need_rows_B_A.columns)
for i in range(len(_columns)):
    _columns[i] = _columns[i][:-2]
need_rows_B_A.columns = _columns

_columns = list(need_rows_B_B.columns)
for i in range(len(_columns)):
    _columns[i] = _columns[i][:-2]
need_rows_B_B.columns = _columns

In [13]:
print('A A', need_rows_A_A.columns,
      '\n\nA B', need_rows_A_B.columns,
      '\n\nB A', need_rows_B_A.columns,
      '\n\nB B', need_rows_B_B.columns)

A A Index(['P510_expense', 'Cu_AT501', 'Cd_AT501', 'Zn_AT501', 'temp', 'pH',
       'Cu_AT502', 'Cd_AT502', 'Zn_AT502', 'auto_W503'],
      dtype='object') 

A B Index(['P510_expense', 'Cu_AT501', 'Cd_AT501', 'Zn_AT501', 'temp', 'pH',
       'Cu_AT502', 'Cd_AT502', 'Zn_AT502', 'auto_W503'],
      dtype='object') 

B A Index(['P510_expense', 'Cu_AT501', 'Cd_AT501', 'Zn_AT501', 'temp', 'pH',
       'Cu_AT502', 'Cd_AT502', 'Zn_AT502', 'auto_W503'],
      dtype='object') 

B B Index(['P510_expense', 'Cu_AT501', 'Cd_AT501', 'Zn_AT501', 'temp', 'pH',
       'Cu_AT502', 'Cd_AT502', 'Zn_AT502', 'auto_W503'],
      dtype='object')


In [14]:
frames = [need_rows_A_A, need_rows_A_B, need_rows_B_A, need_rows_B_B]

result = pd.concat(frames)
result.index = range(0, len(result))

In [15]:
result

,P510_expense,Cu_AT501,Cd_AT501,Zn_AT501,temp,pH,Cu_AT502,Cd_AT502,Zn_AT502,auto_W503
0,0.02,412.7,517.7,127.5,36.1,5.3,27.79,343.00,132.5,178.90
1,0.00,412.7,517.7,127.5,36.1,5.3,27.79,343.00,132.5,178.90
2,0.08,412.7,517.7,127.5,36.1,5.3,27.79,343.00,132.5,178.90
3,0.07,412.7,517.7,127.5,36.1,5.3,27.79,343.00,132.5,178.90
4,0.04,412.7,517.7,127.5,36.1,5.3,27.79,343.00,132.5,178.90
...,...,...,...,...,...,...,...,...,...,...
175355,250.00,133.4,452.0,148.6,51.8,5.2,0.00,55.42,149.7,37.76
175356,248.00,133.4,452.0,148.6,51.8,5.2,0.00,55.42,149.7,37.64
175357,250.00,133.4,452.0,148.6,51.9,5.2,0.00,55.42,149.7,37.81
175358,250.00,133.4,452.0,148.6,51.9,5.2,0.00,55.42,149.7,37.89


In [16]:
result.index

RangeIndex(start=0, stop=175360, step=1)

In [17]:
result.head()

,P510_expense,Cu_AT501,Cd_AT501,Zn_AT501,temp,pH,Cu_AT502,Cd_AT502,Zn_AT502,auto_W503
0,0.02,412.7,517.7,127.5,36.1,5.3,27.79,343.0,132.5,178.9
1,0.00,412.7,517.7,127.5,36.1,5.3,27.79,343.0,132.5,178.9
2,0.08,412.7,517.7,127.5,36.1,5.3,27.79,343.0,132.5,178.9
3,0.07,412.7,517.7,127.5,36.1,5.3,27.79,343.0,132.5,178.9
4,0.04,412.7,517.7,127.5,36.1,5.3,27.79,343.0,132.5,178.9


In [18]:
pd.isnull(result).any()

P510_expense    False
Cu_AT501         True
Cd_AT501         True
Zn_AT501         True
temp             True
pH               True
Cu_AT502         True
Cd_AT502         True
Zn_AT502         True
auto_W503       False
dtype: bool

In [19]:
result = result[result['Cu_AT501'].notna()]
result = result[result['Zn_AT502'].notna()]#result = result[result['EPS'].notna()]

In [20]:
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split

# import statsmodels.api as sm
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from sklearn import preprocessing

In [21]:
# x = result.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# new_result = pd.DataFrame(x_scaled, columns = result.columns)


# result_log = pd.DataFrame()
# for i in result.columns:
#     y_log = np.log(result[i] + math.e)
#     result_log[i] = y_log

In [22]:
# # Экспонента
# for col in result_log.columns:
#     plt.figure(figsize=(10, 6))
#     plt.hist(result_log[col], bins=50, ec='black', color='#2196f3')
#     plt.xlabel(col)
#     plt.ylabel('Count')
#     plt.show

In [23]:
# # Без обработки
# for col in result.columns:
#     plt.figure(figsize=(10, 6))
#     plt.hist(result[col], bins=50, ec='black', color='#2196f3')
#     plt.xlabel(col)
#     plt.ylabel('Count')
#     plt.show

In [24]:
# for col in new_result.columns:
#     plt.figure(figsize=(10, 6))
#     plt.hist(new_result[col], bins=50, ec='black', color='#2196f3')
#     plt.xlabel(col)
#     plt.ylabel('Count')
#     plt.show

In [25]:
# from sklearn.linear_model import LinearRegression

# target_column = result_log['Cd_AT502']
# feat = result_log.drop('Cu_AT502', axis=1)
# feat = feat.drop('Cd_AT502', axis=1)
# feat = feat.drop('Zn_AT502', axis=1)

# X_train, X_test, y_train, y_test = train_test_split(feat, target_column, test_size=0.2)

In [26]:
# regr = LinearRegression()
# regr.fit(X_train, y_train)

# print('Training data r-squader:', regr.score(X_train, y_train))
# print('Test data r-squader:', regr.score(X_test, y_test))

# print('Intercept', regr.intercept_)
# pd.DataFrame(data=regr.coef_, index=X_train.columns, columns=['coef'])

In [27]:
# # Сбор всех экселей
# import os

# IN_PATH = r"C:\Users\Nikita\Desktop\HAKATON\in"
# OUT_PATH = r"C:\Users\Nikita\Desktop\HAKATON\out.csv"
# file_names = os.listdir(IN_PATH)

# files = []

# for file_name in file_names:
#     file_path = os.path.join(IN_PATH, file_name)
#     file = pd.read_excel(file_path)
    
#     file.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
#     file.drop(labels = ' ', axis = 1, inplace = True)
#     file.drop(labels = 0, axis = 0, inplace = True)
    
#     files.append(file)

# result = pd.concat(files)
# result.columns = df.columns

# result.to_csv(OUT_PATH, index=False)

In [28]:
from keras.datasets import boston_housing
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate

In [29]:
# Логарифмирование
# result_log = pd.DataFrame()
# for i in result.columns:
#     y_log = np.log(result[i] + math.e)
#     result_log[i] = y_log
# result = result_log
    
target_column = result['Cu_AT502']
feat = result.drop('Cu_AT502', axis=1)
feat = feat.drop('Cd_AT502', axis=1)
feat = feat.drop('Zn_AT502', axis=1)
x_train, x_test, y_train, y_test = train_test_split(feat, target_column, test_size=0.2)

In [30]:
x_train, x_test, y_train, y_test = x_train.values, x_test.values, y_train.values, y_test.values

In [31]:
# Среднее значение
mean = x_train.mean(axis=0)
# Стандартное отклонение
std = x_train.std(axis=0)
x_train -= mean
x_train /= std
x_test -= mean
x_test /= std

In [50]:
# model = Sequential()

in_expense = Input(shape=(1,))
in_Cu = Input(shape=(1,))
in_Cd = Input(shape=(1,))
in_Zn = Input(shape=(1,))
in_Temp = Input(shape=(1,))
in_Ph = Input(shape=(1,))
in_Auto = Input(shape=(1,))

expense = Dense(16, kernel_initializer='normal', activation="relu")(in_expense)
expense = Dense(12, activation="relu")(expense)
expense = Dense(4, activation="relu")(expense)
expense = Model(inputs=in_expense, outputs=expense)

Cu = Dense(16, activation="relu")(in_Cu)
Cu = Dense(12, activation="relu")(Cu)
Cu = Dense(4, activation="relu")(Cu)
Cu = Model(inputs=in_Cu, outputs=Cu)

Cd = Dense(16, activation="relu")(in_Cd)
Cd = Dense(12, activation="relu")(Cd)
Cd = Dense(4, activation="relu")(Cd)
Cd = Model(inputs=in_Cd, outputs=Cd)

Zn = Dense(16, activation="relu")(in_Zn)
Zn = Dense(12, activation="relu")(Zn)
Zn = Dense(4, activation="relu")(Zn)
Zn = Model(inputs=in_Zn, outputs=Zn)

Temp = Dense(16, activation="relu")(in_Temp)
Temp = Dense(12, activation="relu")(Temp)
Temp = Dense(4, activation="relu")(Temp)
Temp = Model(inputs=in_Temp, outputs=Temp)

Ph = Dense(16, activation="relu")(in_Ph)
Ph = Dense(12, activation="relu")(Ph)
Ph = Dense(4, activation="relu")(Ph)
Ph = Model(inputs=in_Ph, outputs=Ph)

Auto = Dense(16, activation="relu")(in_Auto)
Auto = Dense(12, activation="relu")(Auto)
Auto = Dense(4, activation="relu")(Auto)
Auto = Model(inputs=in_Auto, outputs=Auto)

combined = concatenate([expense.output, Cu.output, Cd.output, Zn.output, Temp.output, Ph.output, Auto.output])

z = Dense(256, activation="relu")(combined)
z = Dense(128, activation="relu")(z)
z = Dense(64, activation="relu")(z)
z = Dense(1, activation="linear")(z)

model = Model(inputs=[[expense.input, Cu.input, Cd.input, Zn.input, Temp.input, Ph.input, Auto.input]], outputs=z)




# define two sets of inputs
# inputA = Input(shape=(4,))
# inputB = Input(shape=(3,))
 
# # the first branch operates on the first input
# x = Dense(256, activation="relu")(inputA)
# x = Dense(128, activation="relu")(x)
# x = Model(inputs=inputA, outputs=x)
 
# the second branch opreates on the second input
# y = Dense(256, activation="relu")(inputB)
# #y = Dense(32, activation="relu")(y)
# y = Dense(128, activation="relu")(y)
# y = Model(inputs=inputB, outputs=y)
 
# combine the output of the two branches
# combined = concatenate([x.output, y.output])
 
# apply a FC layer and then a regression prediction on the
# combined outputs
# z = Dense(64, activation="relu")(combined)
# z = Dense(1, activation="linear")(z)
 
# our model will accept the inputs of the two branches and
# then output a single value
# model = Model(inputs=[x.input, y.input], outputs=z)



# model.add(Dense(128, activation='relu', input_shape=(21,)))
# model.add(Dense(64, activation='relu', input_shape=(128,)))
# model.add(Dense(32, activation='relu', input_shape=(64,)))
# model.add(Dense(16, activation='relu', input_shape=(32,)))
# model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [45]:
model.fit([x_train[:,0:1], x_train[:,1:2], x_train[:,2:3], x_train[:,3:4], x_train[:,4:5], x_train[:,5:6], x_train[:,6:7]], y_train, epochs=750, batch_size=1000, verbose=2)

Epoch 1/750
141/141 - 2s - loss: 77.5519 - mae: 4.6814 - 2s/epoch - 11ms/step
Epoch 2/750
141/141 - 1s - loss: 76.1823 - mae: 4.6385 - 1s/epoch - 10ms/step
Epoch 3/750
141/141 - 1s - loss: 80.2186 - mae: 4.7937 - 1s/epoch - 11ms/step
Epoch 4/750
141/141 - 1s - loss: 76.1599 - mae: 4.6438 - 1s/epoch - 11ms/step
Epoch 5/750
141/141 - 1s - loss: 76.6924 - mae: 4.6544 - 1s/epoch - 11ms/step
Epoch 6/750
141/141 - 1s - loss: 78.8225 - mae: 4.7349 - 1s/epoch - 10ms/step
Epoch 7/750
141/141 - 1s - loss: 77.5836 - mae: 4.7021 - 1s/epoch - 11ms/step
Epoch 8/750
141/141 - 1s - loss: 76.2162 - mae: 4.6419 - 1s/epoch - 10ms/step
Epoch 9/750
141/141 - 1s - loss: 78.4325 - mae: 4.7140 - 1s/epoch - 11ms/step
Epoch 10/750
141/141 - 1s - loss: 82.4859 - mae: 4.8139 - 1s/epoch - 10ms/step
Epoch 11/750
141/141 - 2s - loss: 81.4911 - mae: 4.8341 - 2s/epoch - 11ms/step
Epoch 12/750
141/141 - 1s - loss: 78.4953 - mae: 4.7503 - 1s/epoch - 11ms/step
Epoch 13/750
141/141 - 1s - loss: 76.5323 - mae: 4.6835 - 1s/

In [37]:
x_train

array([[-0.72714961,  0.64508361, -0.56909567, ...,  0.19705373,
         0.19128042,  0.30341643],
       [ 0.5716388 ,  0.78528253,  0.36442665, ...,  0.33010818,
         0.00376415,  1.30076335],
       [-1.35489735,  0.35524232,  0.0987163 , ..., -0.30855322,
         0.37879669, -0.32348735],
       ...,
       [ 0.39846701,  0.82450917,  1.21712453, ...,  0.83571513,
         0.00376415, -0.71739748],
       [-0.22928072, -0.45980011, -0.03868526, ..., -0.86738194,
        -0.18375213, -0.88166639],
       [-0.70550314, -0.22371384, -0.50443611, ...,  0.17044283,
         0.19128042, -0.73583583]])

In [35]:
x_train[:,6:7]

array([[ 0.30341643],
       [ 1.30076335],
       [-0.32348735],
       ...,
       [-0.71739748],
       [-0.88166639],
       [-0.73583583]])

In [46]:
mse, mae = model.evaluate([x_test[:,0:1], x_test[:,1:2], x_test[:,2:3], x_test[:,3:4], x_test[:,4:5], x_test[:,5:6], x_test[:,6:7]], y_test, verbose=0)

In [47]:
print("Средняя абсолютная ошибка:", mae)

Средняя абсолютная ошибка: 4.490440845489502


In [48]:
x_train.shape

(140283, 7)

In [49]:
pred = model.predict([x_test[:,0:1], x_test[:,1:2], x_test[:,2:3], x_test[:,3:4], x_test[:,4:5], x_test[:,5:6], x_test[:,6:7]])
for i in range(0, pred.shape[0]):
    print("Предсказание:", pred[i][0], ", правильно:", y_test[i])
# 

1096/1096 [==============================] - 3s 3ms/step
Предсказание: 2.4616609 , правильно: 0.0
Предсказание: 54.08378 , правильно: 55.86
Предсказание: 78.102066 , правильно: 83.56
Предсказание: 16.91321 , правильно: 17.14
Предсказание: 43.956882 , правильно: 22.4
Предсказание: 135.89307 , правильно: 129.4
Предсказание: 66.43742 , правильно: 58.94
Предсказание: 20.9589 , правильно: 25.77
Предсказание: 0.9455227 , правильно: 0.0
Предсказание: 52.623466 , правильно: 44.5
Предсказание: 31.690365 , правильно: 38.98
Предсказание: 30.697704 , правильно: 24.74
Предсказание: 33.213867 , правильно: 34.17
Предсказание: 22.59439 , правильно: 21.32
Предсказание: 51.678726 , правильно: 54.77
Предсказание: 36.43108 , правильно: 38.11
Предсказание: 25.650137 , правильно: 18.54
Предсказание: 26.339245 , правильно: 23.02
Предсказание: -0.32854396 , правильно: 0.0
Предсказание: 167.10037 , правильно: 173.2
Предсказание: 100.99534 , правильно: 108.0
Предсказание: 42.215057 , правильно: 46.15
Предсказан

In [ ]:
pred.shape

In [ ]:
# pred_exp = np.exp(pred) - math.e
# y_test_exp = np.exp(y_test) - math.e
# for i in range(0, pred_exp.shape[0]):
#     print("Предсказание:", pred_exp[i][0], ", правильно:", y_test_exp[i])

In [ ]:
# evaluate model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [51]:
tf.keras.models.save_model(model, 'HAKATON.m')

INFO:tensorflow:Assets written to: HAKATON.m\assets
